In [1]:
#initial imports
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import numpy as np 
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
import numpy as np

In [2]:
#read in incidence csv from github

mortality_df = pd.read_csv("https://raw.githubusercontent.com/robyndook/Cancer_Treatment_Centers_California/main/Resources/Age_Mortality_Cancer_DB.csv")                         
#get rid of commas in columns
mortality_df = mortality_df.replace(',','', regex=True)
mortality_df.head()     

,average_annual_count,primary_state_name,cancer,race_ethnicity,sex,age,age_adjusted_rate,county_state,index_of_medical_underservice_score,popestimate2015,popestimate2016,popestimate2017,popestimate2018,popestimate2019,abbrv,median_household_income_2015,median_household_income_2016,median_household_income_2017,median_household_income_2018,median_household_income_2019
0,4,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,38.8,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
1,48,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,50+,606.5,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
2,18,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<65,84.7,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
3,35,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,65+,963.0,Marengo County AL,54.0,19766,19525,19396,19056,18863,AL,35389.0,34794.0,37469.0,37733.0,38838.0
4,9,Alabama,All Cancer Sites,All Races (includes Hispanic),All Sexes,<50,30.2,Jackson County AL,53.0,52195,51988,51828,51621,51626,AL,40201.0,41407.0,42658.0,41929.0,44322.0


In [3]:
#check dtypes
mortality_df.dtypes

average_annual_count                     int64
primary_state_name                      object
cancer                                  object
race_ethnicity                          object
sex                                     object
age                                     object
age_adjusted_rate                       object
county_state                            object
index_of_medical_underservice_score    float64
popestimate2015                          int64
popestimate2016                          int64
popestimate2017                          int64
popestimate2018                          int64
popestimate2019                          int64
abbrv                                   object
median_household_income_2015           float64
median_household_income_2016           float64
median_household_income_2017           float64
median_household_income_2018           float64
median_household_income_2019           float64
dtype: object

In [4]:
#get count of rows
mortality_df.count()

average_annual_count                   8170
primary_state_name                     8170
cancer                                 8170
race_ethnicity                         8170
sex                                    8170
age                                    8170
age_adjusted_rate                      8170
county_state                           8170
index_of_medical_underservice_score    7091
popestimate2015                        8170
popestimate2016                        8170
popestimate2017                        8170
popestimate2018                        8170
popestimate2019                        8170
abbrv                                  8152
median_household_income_2015           8152
median_household_income_2016           8152
median_household_income_2017           8152
median_household_income_2018           8152
median_household_income_2019           8152
dtype: int64

In [5]:
#get count of values for age
mortality_df["age"].value_counts()

50+    2420
65+    2406
<65    2198
<50    1146
Name: age, dtype: int64

In [6]:
#get count of rows
mortality_df.count()

average_annual_count                   8170
primary_state_name                     8170
cancer                                 8170
race_ethnicity                         8170
sex                                    8170
age                                    8170
age_adjusted_rate                      8170
county_state                           8170
index_of_medical_underservice_score    7091
popestimate2015                        8170
popestimate2016                        8170
popestimate2017                        8170
popestimate2018                        8170
popestimate2019                        8170
abbrv                                  8152
median_household_income_2015           8152
median_household_income_2016           8152
median_household_income_2017           8152
median_household_income_2018           8152
median_household_income_2019           8152
dtype: int64

In [7]:
#keep only columns with 65+ and <65
mortality_df_65 = mortality_df[mortality_df["age"] != "50+"] 
mortality_df_65 = mortality_df_65[mortality_df_65["age"] != "<50"] 
mortality_df_65["age"].value_counts()

65+    2406
<65    2198
Name: age, dtype: int64

In [8]:
#count rows
mortality_df_65.count()

average_annual_count                   4604
primary_state_name                     4604
cancer                                 4604
race_ethnicity                         4604
sex                                    4604
age                                    4604
age_adjusted_rate                      4604
county_state                           4604
index_of_medical_underservice_score    3998
popestimate2015                        4604
popestimate2016                        4604
popestimate2017                        4604
popestimate2018                        4604
popestimate2019                        4604
abbrv                                  4595
median_household_income_2015           4595
median_household_income_2016           4595
median_household_income_2017           4595
median_household_income_2018           4595
median_household_income_2019           4595
dtype: int64

In [9]:
#create target 
y = mortality_df_65[['age_adjusted_rate']]

#change dtype to numeric
y['age_adjusted_rate'] = y['age_adjusted_rate'].astype(float)
y.head()

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,age_adjusted_rate
2,84.7
3,963.0
6,80.3
7,986.9
10,69.2


In [13]:
#create feautures variable holding age
X = mortality_df_65[["age"]]

In [14]:
#create dummy variables for age
X = pd.get_dummies(X, columns=["age"])
X.head()

,age_65+,age_<65
2,0,1
3,1,0
6,0,1
7,1,0
10,0,1


In [15]:
#get shape of target array
y.shape

(4604, 1)

In [16]:
#get shape of feautures array
X.shape

(4604, 2)

In [17]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
#create linear regression model using sklearn
model = LinearRegression()

In [19]:
#fit the model with training data
regression_age_inc_mortality = model.fit(X_train, y_train)

In [20]:
#Create predictions array using linear regression
y_pred = regression_age_inc_mortality.predict(X_test)
y_pred

array([[923.02868897],
       [923.02868897],
       [ 56.65766871],
       ...,
       [923.02868897],
       [923.02868897],
       [923.02868897]])

In [21]:
#get R2 value from linear regression
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.9524862483805986

In [22]:
#run regression again with statsmodels
#use statsmodels to add constant to X
#add constant to X
X1 = sm.add_constant(X)

/Users/hankbowman/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [23]:
#fit linear regression model
model = sm.OLS(y, X1).fit()

In [24]:
#view model summary with statsmodels
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:      age_adjusted_rate   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.947
Method:                 Least Squares   F-statistic:                 8.237e+04
Date:                Fri, 03 Jun 2022   Prob (F-statistic):               0.00
Time:                        15:11:53   Log-Likelihood:                -27842.
No. Observations:                4604   AIC:                         5.569e+04
Df Residuals:                    4602   BIC:                         5.570e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        327.0383      1.007    324.774      0.0

In [25]:
#fit the model with training data
#decision tree regression
regressor_DT = DecisionTreeRegressor()
regressor_DT.fit(X_train, y_train)

DecisionTreeRegressor()

In [26]:
#predict
y_pred = regressor_DT.predict(X_test)
y_pred

array([923.02868897, 923.02868897,  56.65766871, ..., 923.02868897,
       923.02868897, 923.02868897])

In [27]:
#calculate r2 score
r2_score(y_test, y_pred)

0.9524862483805987